### Ejercicio 2

Julián Cámara Miró

Para probar, vamos a definir una función que aplique n rotaciones a un unico qubit, cada una de pi/2**i rotaciones.

In [94]:
import cirq
from math import pi

def n_rotaciones(n: int):

    a = cirq.NamedQubit('a')
    circuit = cirq.Circuit()

    for i in range(1, n+1):
        circuit.append(cirq.rz(pi/2**i)(a))

    return circuit

print(n_rotaciones(3))

a: ───Rz(0.5π)───Rz(0.25π)───Rz(0.125π)───


In [95]:
import numpy as np
np.set_printoptions(precision=2)

sim = cirq.Simulator()

circuit = n_rotaciones(1)

vector_final = sim.simulate(circuit).final_state_vector
print(vector_final)

[0.71-0.71j 0.  +0.j  ]


In [48]:
import cirq

qubits = cirq.LineQubit.range(4)
circuit = cirq.Circuit(cirq.H(qubits[0]), cirq.CZ(qubits[1], qubits[0]), cirq.CZ(qubits[2], qubits[0])**0.5, cirq.CZ(qubits[3], qubits[0])**0.25,
                       cirq.H(qubits[1]), cirq.CZ(qubits[2], qubits[1]), cirq.CZ(qubits[3], qubits[1])**0.5,
                       cirq.H(qubits[2]), cirq.CZ(qubits[3], qubits[2]),
                       cirq.H(qubits[3]))
                    #    cirq.measure(qubits[0], qubits[1], qubits[2], qubits[3]))

print(circuit)

              ┌──────┐   ┌───────┐   ┌──────┐
0: ───H───@────@──────────@─────────────────────────────
          │    │          │
1: ───────@────┼────H─────┼─────@─────@─────────────────
               │          │     │     │
2: ────────────@^0.5──────┼─────@─────┼────H────@───────
                          │           │         │
3: ───────────────────────@^0.25──────@^0.5─────@───H───
              └──────┘   └───────┘   └──────┘


In [42]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.1f}'.format})

sim = cirq.Simulator()

vector_final = sim.simulate(circuit).final_state_vector
print(np.real(vector_final))

[ 0.2  0.0]
